In [1]:
import os, gzip, binascii, struct, numpy
from six.moves.urllib.request import urlretrieve
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


def fully_conntected_activation(input,size_in,size_out):
    W = tf.Variable(tf.truncated_normal([size_in, size_out],stddev=0.1))
    b = tf.Variable(tf.truncated_normal([size_out], stddev=0.1))
    activation = tf.nn.relu(tf.matmul(input,W)+b)
    
    return activation

def fully_conntected(input,size_in,size_out):
    W = tf.Variable(tf.truncated_normal([size_in, size_out],stddev=0.1))
    b = tf.Variable(tf.truncated_normal([size_out], stddev=0.1))
    activation = tf.matmul(input,W)+b
    
    return activation

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

IMAGE_SIZE = 28
HIDDEN_LAYER = 300
NUM_LABELS = 10

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
def mnist_run():

    x = tf.placeholder(tf.float32, [None, IMAGE_SIZE * IMAGE_SIZE])
    y_true = tf.placeholder(tf.float32,[None, NUM_LABELS])

    hidden = fully_conntected_activation(x,IMAGE_SIZE * IMAGE_SIZE, HIDDEN_LAYER)
    output = fully_conntected(hidden,HIDDEN_LAYER,NUM_LABELS)

    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=output))
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

    # test the model
    correct_prediction = tf.equal(tf.argmax(output,1), tf.argmax(y_true,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    sess = tf.InteractiveSession()
    tf.global_variables_initializer().run()

    # Training
    for i in range(1000):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        l,_,a = sess.run([cross_entropy, train_step, accuracy], 
                         feed_dict={x: batch_xs, y_true: batch_ys})
        #if i%100 == 0 or (i < 100 and i % 10 == 0):
        #    print(str(i)+': loss: '+str(l)+' accuracy: '+str(a))
    # print  l, a
    return l, a

In [4]:
%load_ext line_profiler
%load_ext memory_profiler

In [35]:
#%timeit -n 5 mnist_run() 
%time mnist_run() 

0.0777463 0.976562
CPU times: user 28.4 s, sys: 737 ms, total: 29.2 s
Wall time: 6.83 s


(0.077746257, 0.9765625)

In [6]:
%memit -r 2 mnist_run()

peak memory: 828.62 MiB, increment: 172.55 MiB


In [7]:
def test(N=10):
    losses = []; accus = []
    for i in range(N):
        l, a = mnist_run()
        losses.append(l); accus.append(a)
    print "Loss: %.3f (%.3f); Accuracy: %.3f (%.3f)" % \
        (numpy.mean(losses), numpy.std(losses), numpy.mean(accus), numpy.std(accus))
        
test()